In [1]:
#Fusión de funtes de video
import numpy as np
from tkinter import Tk, filedialog, Button, Label, Frame, Canvas, font, Scale, HORIZONTAL
from PIL import Image, ImageTk
from ultralytics import YOLO
from tkinter import LabelFrame
import cv2
import threading
import time

try:
    from ultralytics import YOLO
except ModuleNotFoundError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics"])
    from ultralytics import YOLO

model = YOLO('C:/PS/Documentos/Mimodelo/runs/detect/yoloPS/weights/best.pt')

polygons = []
polygon_video_indices = []
current_polygon = []
contour_masks = []
polygon_index = 0
polygon_boxes = []
current_mask_index = None

base_image = None
resized_image = None
canvas_width = 800
canvas_height = 700
tk_image_ref = None
canvas_widget = None
image_status_label = None
drawing_status_label = None

video_paths = ["", "", ""]
caps = [None, None, None]
is_playing = False
status_labels = []

frame_buffers = [[], [], []]
frame_buffer_size = 3

def apply_clahe_color(frame, apply_hsv=False, low_quality=False):
    if low_quality:
        frame = cv2.resize(frame, (320, 240), interpolation=cv2.INTER_LINEAR)

    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l2 = clahe.apply(l)
    lab = cv2.merge((l2, a, b))
    frame = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    if apply_hsv:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower = np.array([h_low_slider.get(), s_low_slider.get(), v_low_slider.get()])
        upper = np.array([h_high_slider.get(), s_high_slider.get(), v_high_slider.get()])
        mask = cv2.inRange(hsv, lower, upper)
        frame = cv2.bitwise_and(frame, frame, mask=mask)
    return frame


def reset_state():
    global polygons, current_polygon, contour_masks, polygon_boxes, current_mask_index, polygon_video_indices
    polygons.clear()
    current_polygon.clear()
    contour_masks.clear()
    polygon_boxes.clear()
    polygon_video_indices.clear()
    current_mask_index = None


def reset_videos():
    global video_paths, caps, frame_buffers, is_playing
    is_playing = False
    for i in range(2):
        #if caps[i] is not None:
        #    caps[i].release()
        #caps[i] = None
        video_paths[i] = None
        frame_buffers[i].clear()
        status_labels[i].config(text=f"Espera Video {i+1}...", fg="gray")
        
def select_image():
    global base_image, resized_image, tk_image_ref, is_playing
    reset_state()
    #reset_videos()  
   
    # Seleccionar imagen
    path = filedialog.askopenfilename(title="Selecciona la imagen", filetypes=[("Images", "*.jpg *.png")])
    if not path:
        image_status_label.config(text="Espera imagen", fg="orange")
        return

   
    canvas_widget.delete("all")
    drawing_status_label.config(text="", fg="blue")
    image_status_label.config(text="Cargando imagen...", fg="orange")

    # Leer imagen original
    base_image = cv2.imread(path)
    h, w = base_image.shape[:2]

    # Redimensionar proporcionalmente sin deformar
    scale = min(canvas_width / w, canvas_height / h)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(base_image, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Crear lienzo blanco y centrar la imagen en él
    padded_image = np.full((canvas_height, canvas_width, 3), 255, dtype=np.uint8)
    x_offset = (canvas_width - new_w) // 2
    y_offset = (canvas_height - new_h) // 2
    padded_image[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized
    resized_image = padded_image.copy()

    # Detección de marcos con YOLO
    results = model(resized_image)[0]
    contour_masks.clear()
    for box in results.boxes.xyxy.cpu().numpy():
        x1, y1, x2, y2 = map(int, box)
        mask = np.zeros((canvas_height, canvas_width), dtype=np.uint8)
        cv2.rectangle(mask, (x1, y1), (x2, y2), 255, -1)
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        contour_masks.append(mask)

    drawing_status_label.config(text=f"Marcos detectados: {len(contour_masks)}", fg="green")

    # Dibujar contornos sobre la imagen
    overlay = resized_image.copy()
    for mask in contour_masks:
        overlay = cv2.addWeighted(overlay, 0.7, overlay, 0.3, 0)
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(overlay, cnts, -1, (0, 255, 0), 2)

    # Convertir a formato tkinter
    img_rgb = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    tk_img = ImageTk.PhotoImage(Image.fromarray(img_rgb))
    tk_image_ref = tk_img

    canvas_widget.config(width=canvas_width, height=canvas_height)
    canvas_widget.delete("all")
    canvas_widget.create_image(0, 0, anchor="nw", image=tk_img)
    canvas_widget.image = tk_img

    image_status_label.config(text="Imagen cargada ✔", fg="green")


def draw_selection():
    global current_polygon, polygon_index, current_mask_index
    if resized_image is None or not contour_masks:
        drawing_status_label.config(text="Carga imagen primero!", fg="red")
        return
    current_polygon = []
    current_mask_index = None
    canvas_widget.unbind("<Button-1>"); canvas_widget.unbind("<Button-3>")
    canvas_widget.bind("<Button-1>", on_click); canvas_widget.bind("<Button-3>", on_right_click)
    drawing_status_label.config(text="Dibuja polígono (clic izquierdo + clic derecho)", fg="blue")

def on_click(event):
    global current_polygon, current_mask_index
    x, y = event.x, event.y
    if y >= resized_image.shape[0] or x >= resized_image.shape[1]:
        drawing_status_label.config(text="Fuera del marco", fg="red")
        return

    match_index = None
    for i, mask in enumerate(contour_masks):
        if y < mask.shape[0] and x < mask.shape[1] and mask[y, x] == 255:
            match_index = i
            break

    if match_index is None:
        drawing_status_label.config(text="Fuera del marco", fg="red")
        return

    if current_mask_index is None:
        if any(pb == i for pb, i in zip(polygon_boxes, range(len(contour_masks))) if i == match_index):
            drawing_status_label.config(text="Ya hay un polígono en este marco", fg="red")
            return
        current_mask_index = match_index
    elif current_mask_index != match_index:
        drawing_status_label.config(text="Solo se permite un polígono por marco", fg="red")
        return

    drawing_status_label.config(text="")
    current_polygon.append((x, y))
    r = 3
    canvas_widget.create_oval(x - r, y - r, x + r, y + r, fill="red")
    if len(current_polygon) > 1:
        canvas_widget.create_line(*current_polygon[-2], x, y, fill="blue", width=2)

def on_right_click(event):
    global polygon_index
    if len(current_polygon) < 3:
        drawing_status_label.config(text="Se necesitan ≥3 puntos", fg="red")
        return

    polygon_index += 1
    drawing_status_label.config(text=f"Polígono #{polygon_index} creado", fg="green")
    canvas_widget.unbind("<Button-1>"); canvas_widget.unbind("<Button-3>")
    polygons.append(current_polygon.copy())
    polygon_boxes.append(cv2.boundingRect(np.array(current_polygon)))
    assigned_idx = (polygon_index - 1) % 3
    polygon_video_indices.append(assigned_idx)
    current_polygon.clear()

def select_camara(index):
    if caps[index]:
        caps[index].release()
    caps[index] = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    if caps[index].isOpened():
        status_labels[index].config(text="Cámara activada ✔", fg="green")
    else:
        status_labels[index].config(text="Error con la cámara ✖", fg="red")

def select_video(index):
    path = filedialog.askopenfilename(title=f"Selecciona el video #{index}", filetypes=[("Videos", "*.mp4 *.avi")])
    if path:
        video_paths[index] = path
        caps[index] = cv2.VideoCapture(path)
        if caps[index].isOpened():
            status_labels[index].config(text=f"Video {index} cargado ✔", fg="green")
        else:
            status_labels[index].config(text=f"Error al cargar Video {index} ✖", fg="red")
    else:
        status_labels[index].config(text=f"Carga de Video {index} cancelada ✖", fg="orange")

def reproducir_videos():
    global is_playing
    prev_time = time.time()
    while is_playing:
        if resized_image is None or not polygons:
            time.sleep(0.01)
            continue

        frame_img = resized_image.copy()

        for i, poly in enumerate(polygons):
            if i >= len(polygon_video_indices):
                continue

            vid_idx = polygon_video_indices[i]
            if vid_idx >= len(caps):
                continue

            cap = caps[vid_idx]

            if cap is None or not cap.isOpened():
                if vid_idx == 0:
                    caps[0] = cv2.VideoCapture(0, cv2.CAP_DSHOW)
                    cap = caps[0]
                    if not cap.isOpened():
                        continue
                    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
                    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
                else:
                    continue

            ret, frame = cap.read()

            if not ret or frame is None:
                if vid_idx == 0:
                    cap.release()
                    caps[0] = cv2.VideoCapture(0, cv2.CAP_DSHOW)
                    cap = caps[0]
                    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
                    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
                    ret, frame = cap.read()
                    if not ret or frame is None:
                        continue
                else:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                    continue

            apply_hsv = (vid_idx == 0)
            frame = apply_clahe_color(frame, apply_hsv, low_quality=False)

            poly_np = np.array(poly, dtype=np.int32)
            x, y, w, h = cv2.boundingRect(poly_np)
            frame_resized = cv2.resize(frame, (w, h), interpolation=cv2.INTER_LANCZOS4)

            mask = np.zeros((canvas_height, canvas_width), dtype=np.uint8)
            cv2.fillPoly(mask, [poly_np], 255)
            alpha = mask.astype(np.float32) / 255.0
            alpha = cv2.merge([alpha, alpha, alpha])

            insert = np.zeros_like(resized_image)
            h_limit = min(h, resized_image.shape[0] - y)
            w_limit = min(w, resized_image.shape[1] - x)

            insert[y:y + h_limit, x:x + w_limit] = frame_resized[:h_limit, :w_limit]
            alpha = alpha[y:y + h_limit, x:x + w_limit]
            base_crop = frame_img[y:y + h_limit, x:x + w_limit]

            blended = (alpha * insert[y:y + h_limit, x:x + w_limit] + (1 - alpha) * base_crop).astype(np.uint8)
            frame_img[y:y + h_limit, x:x + w_limit] = blended

        img_rgb = cv2.cvtColor(frame_img, cv2.COLOR_BGR2RGB)
        tk_img = ImageTk.PhotoImage(Image.fromarray(img_rgb))
        canvas_widget.create_image(0, 0, anchor="nw", image=tk_img)
        canvas_widget.image = tk_img
        canvas_widget.update()

        elapsed = time.time() - prev_time
        wait = max(0.001, 1 / 30.0 - elapsed)
        prev_time = time.time()
        time.sleep(wait)

def on_close():
    global is_playing
    is_playing = False
    for cap in caps:
        if cap:
            cap.release()
    root.destroy()

def key_handler(event):
    global is_playing
    if event.keysym == 'Return':
        if is_playing:
            return

        if not polygons:
            drawing_status_label.config(text="Dibuja al menos un polígono antes de reproducir", fg="red")
            return
        if not any(cap and cap.isOpened() for cap in caps):
            drawing_status_label.config(text="Carga al menos un video", fg="red")
            return

        for cap in caps:
            if cap and cap.isOpened():
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

        is_playing = True
        threading.Thread(target=reproducir_videos, daemon=True).start()

    elif event.keysym == 'Tab':
        is_playing = False


root = Tk()
root.title("Fusión de fuentes de vídeo")
root.geometry("950x700")
root.resizable(False, False)
root.protocol("WM_DELETE_WINDOW", on_close)

default_font = font.Font(size=8)

# Panel izquierdo
from tkinter import Frame, Button, Label

# === Función para agregar efecto hover ===
def add_hover_effect(widget, hover_bg="#d3d3d3", normal_bg=None):
    if normal_bg is None:
        normal_bg = widget.cget("background")

    def on_enter(event):
        widget.config(background=hover_bg)

    def on_leave(event):
        widget.config(background=normal_bg)

    widget.bind("<Enter>", on_enter)
    widget.bind("<Leave>", on_leave)

from tkinter import Frame, Button, Label

# === Función de efecto hover ===
def add_hover_effect(widget, hover_bg="#a9a9a9", normal_bg="#4a90e2"):
    def on_enter(event):
        widget.config(bg=hover_bg)
    def on_leave(event):
        widget.config(bg=normal_bg)
    widget.bind("<Enter>", on_enter)
    widget.bind("<Leave>", on_leave)

# === Estilo base para botones ===
button_style = {
    "font": default_font,
    "bg": "#4a90e2",       # Azul profesional
    "fg": "white",         # Texto blanco
    "relief": "flat",      # Estilo plano
    "anchor": "w",         # Texto alineado a la izquierda
    "padx": 6,            # Espacio interior izquierdo
    "height": 1,            # Altura suficiente para aspecto moderno
    "width": 12,    # <-- fija ancho en caracteres (ajusta según texto)
    "bd": 0
 
}

# === Panel izquierdo ===
left_frame = Frame(root, width=150)
left_frame.pack(side="left", fill="y", padx=5, pady=5)
left_frame.pack_propagate(False)  # <- evita que el frame se deforme con el contenido

# === Botones con emojis coloridos ===
btn_img = Button(left_frame, text="🖼️ Cargar Imagen", command=lambda: select_image(), **button_style)
btn_img.pack(fill="x", pady=2)
add_hover_effect(btn_img)

# === Estado de imagen base ===
image_status_label = Label(left_frame, text="🕓 Espera imagen...", fg="gray", font=default_font, anchor="w", width=30)
image_status_label.pack(pady=(0, 2))

# Botón para Cámara con estado (indice 0)
btn_cam = Button(left_frame, text="📷    Cámara", command=lambda: select_camara(0), **button_style)
btn_cam.pack(fill="x", pady=2)
add_hover_effect(btn_cam)

lbl_cam = Label(left_frame, text="🕓 Esperando cámara...", fg="gray", font=default_font)
lbl_cam.pack(fill="x", pady=(0, 6))
status_labels.append(lbl_cam)

for i in range(2):
    idx = i + 1  # status_labels[1] y status_labels[2]
    btn_video = Button(left_frame, text=f"🎞️Cargar Video {idx}", command=lambda i=idx: select_video(i), **button_style)
    btn_video.pack(fill="x", pady=2)
    add_hover_effect(btn_video)

    lbl_video = Label(left_frame, text=f"🕓 Esperando video {idx}...", fg="gray", font=default_font)
    lbl_video.pack(fill="x", pady=(0, 6))
    status_labels.append(lbl_video)

# === Contenedor de botón y mensaje de dibujo con ancho fijo ===
draw_container = Frame(left_frame, width=3)
draw_container.pack(fill="x", pady=2)
draw_container.pack_propagate(False)

btn_draw = Button(left_frame, text="✏️   Dibujar Polígono", command=draw_selection, **button_style)
btn_draw.pack(fill="x")
add_hover_effect(btn_draw)

# Etiqueta para estado de dibujo
drawing_status_label = Label(
    draw_container,
    text="",
    fg="blue",
    font=default_font,
    width=8,
    wraplength=10,
    anchor="w",
    justify="left"
)
drawing_status_label.pack(pady=(2, 0))
# === Frame contenedor de los sliders HSV ===
hsv_frame = Frame(left_frame, bd=2, relief="groove")
hsv_frame.pack(pady=5)

# Crear el título centrado como Label dentro del Frame
titulo_hsv = Label(hsv_frame, text="🎨 Ajustar filtros HSV", 
                   font=("Arial", 7, "bold"), fg="blue", bg="#F5F2F2")
titulo_hsv.pack(pady=(0, 10))

# === Sliders HSV con colores ===
slider_length = 150
pad_y = 2

h_low_slider = Scale(hsv_frame, from_=0, to=179, orient=HORIZONTAL, label="H Bajo",
                     length=slider_length, font=("Arial", 7), fg="red", highlightcolor="red")
h_low_slider.set(0)
h_low_slider.pack(pady=pad_y)

h_high_slider = Scale(hsv_frame, from_=0, to=179, orient=HORIZONTAL, label="H Alto",
                      length=slider_length, font=("Arial", 7), fg="red", highlightcolor="red")
h_high_slider.set(179)
h_high_slider.pack(pady=pad_y)

s_low_slider = Scale(hsv_frame, from_=0, to=255, orient=HORIZONTAL, label="S Bajo",
                     length=slider_length, font=("Arial", 7), fg="green", highlightcolor="green")
s_low_slider.set(0)
s_low_slider.pack(pady=pad_y)

s_high_slider = Scale(hsv_frame, from_=0, to=255, orient=HORIZONTAL, label="S Alto",
                      length=slider_length, font=("Arial", 7), fg="green", highlightcolor="green")
s_high_slider.set(255)
s_high_slider.pack(pady=pad_y)

v_low_slider = Scale(hsv_frame, from_=0, to=255, orient=HORIZONTAL, label="V Bajo",
                     length=slider_length, font=("Arial", 7), fg="blue", highlightcolor="blue")
v_low_slider.set(0)
v_low_slider.pack(pady=pad_y)

v_high_slider = Scale(hsv_frame, from_=0, to=255, orient=HORIZONTAL, label="V Alto",
                      length=slider_length, font=("Arial", 5), fg="blue", highlightcolor="blue")
v_high_slider.set(255)
v_high_slider.pack(pady=pad_y)

# === Botón para restaurar valores por defecto ===
def reset_hsv():
    h_low_slider.set(0)
    h_high_slider.set(179)
    s_low_slider.set(0)
    s_high_slider.set(255)
    v_low_slider.set(0)
    v_high_slider.set(255)

reset_button = Button(hsv_frame, text="Restaurar HSV", command=reset_hsv, **button_style)
reset_button.pack(pady=1)
add_hover_effect(reset_button)

# Panel derecho para el video y canvas
canvas_widget = Canvas(root, width=canvas_width, height=canvas_height, bg="white")
canvas_widget.pack(side="right", padx=1, pady=1)

root.bind("<Key>", key_handler)

root.mainloop()
